In [9]:
import torch
import torch.nn as nn

In [10]:
class NonSeqCNN(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(128, 128, kernel_size=3, padding=1)

    def forward(self, x):
        out1 = torch.relu(self.conv1(x))
        out2 = torch.relu(self.conv2(out1))
        out3 = torch.relu(self.conv3(out2))
        out4 = torch.relu(self.conv4(out3))
        # Apply a 1x1 convolution to match dimensions for the skip connection
        out2_skip = torch.relu(nn.Conv2d(64, 128, kernel_size=1)(out2))
        # Introduce skip connection by adding outputs of conv2 and conv3
        out4_skip = out4 + out2_skip
        out5 = torch.relu(self.conv5(out4_skip))
        return out5

model = NonSeqCNN()

In [11]:
dummy_input = torch.randn(1, 3, 32, 32)

In [12]:
trace = torch.jit.trace(model, dummy_input)

/home/willian/.local/lib/python3.11/site-packages/torch/jit/_trace.py:1102: TracerWarning: Trace had nondeterministic nodes. Did you forget call .eval() on your model? Nodes:
	%tensor.3 : Float(128, 64, 1, 1, strides=[64, 1, 1, 1], requires_grad=1, device=cpu) = aten::uniform_(%tensor.1, %173, %174, %175) # /home/willian/.local/lib/python3.11/site-packages/torch/nn/init.py:459:0
	%bias.9 : Float(128, strides=[1], requires_grad=1, device=cpu) = aten::uniform_(%tensor, %202, %203, %204) # /home/willian/.local/lib/python3.11/site-packages/torch/nn/init.py:15:0
This may cause errors in trace checking. To disable trace checking, pass check_trace=False to torch.jit.trace()
  _check_trace(
/home/willian/.local/lib/python3.11/site-packages/torch/jit/_trace.py:1102: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 87964 / 131072 (67.1%)
Greatest absolute difference

In [13]:
trace_output = trace(dummy_input)

In [15]:
print(trace.code, type(trace.code))

def forward(self,
    x: Tensor) -> Tensor:
  conv5 = self.conv5
  conv4 = self.conv4
  conv3 = self.conv3
  conv2 = self.conv2
  conv1 = self.conv1
  input = torch.relu((conv1).forward(x, ))
  input0 = torch.relu((conv2).forward(input, ))
  input1 = torch.relu((conv3).forward(input0, ))
  out4 = torch.relu((conv4).forward(input1, ))
  data = torch.empty([128, 64, 1, 1], dtype=None, layout=None, device=torch.device("cpu"), pin_memory=False)
  tensor = torch.detach(data)
  data0 = torch.empty([128], dtype=None, layout=None, device=torch.device("cpu"), pin_memory=False)
  tensor0 = torch.detach(data0)
  tensor1 = torch.uniform_(tensor, -0.12499999999999999, 0.12499999999999999)
  bias = torch.uniform_(tensor0, -0.125, 0.125)
  _0 = torch._convolution(input0, tensor1, bias, [1, 1], [0, 0], [1, 1], False, [0, 0], 1, False, False, True, True)
  out2_skip = torch.relu(_0)
  input2 = torch.add(out4, out2_skip)
  return torch.relu((conv5).forward(input2, ))
 <class 'str'>
